# Leveraging Existing USGS Streamgage Data to Map Flood-Prone Areas
---
Team: Marina Metes (Lead PI), Labeeb Ahmed, Tristan Mohs & Greg Noe
Contact: `mmetes@usgs.gov` or `lahmed@usgs.gov`

## Summary
Knowing where floods occur is crucial for a wide range of stakeholder needs, from protecting people and infrastructure from flood risk, to managing and maintaining healthy riparian and floodplain ecosystems that frequently flood. The goal of this project is to develop automated workflows using open-source software and publicly available data to map flood extent in locations that are unmonitored. The output will be continuous digital representations of floodplains across stream networks for a range of annual exceedance probabilities. The floodplains will also include width and other metrics averaged by cross-section, reach, or watershed. A model to estimate flood extent in unmonitored locations will be calibrated using existing USGS streamgage data and field measurements, and elevation data derived from lidar. This project will develop methods for using streamgage data outside traditional applications and create workflows for more readily accessing intermediate datasets associated with streamgage maintenance that are relevant to floodplain mapping.

## Interactive Document
This jupyter notebook is an interactive document with explanations followed by code snippets and interactive widgets

## Table of Contents:
1. [Python Imports](#python-imports)
[Map and explore lidar collections](#item)

### Python Imports <a name="python-imports"></a>

In [ ]:
from pathlib import Path
from lapis import lapis, noaa_dav, gps_time, flood_stats
import leafmap.leafmap as leafmap
import dataretrieval.nwis as nwis

### Create data folder

In [ ]:
Path("data").mkdir(parents=True, exist_ok=True)

### User Inputs: 
Coordinates (converted to circles), polygon features (shapefiles, gpkg, geojson etc., -- Only single part), or bounding boxes. 

**Note: all input data should be in World Geodetic System 1984 (WGS 84)**

In [ ]:
locations_dict = [
    {
        'name': 'gage id 01650500',
        'data_type': 'coords',
        'value': (-77.0293611, 39.06552778), #longitude, latitude
        'buffer': 500, # in meters -- default value is 50 meters unless specified 
    },
    {
        'name': 'gage id 01651000',
        'data_type': 'coords',
        'value': (-76.96513889, 38.95255556), #longitude, latitude
        'buffer': 500, # in meters -- default value is 50 meters unless specified 
    },
]

Users can also provide bounding boxes or geojson:

Here's an example of passing geojson file that represents boundary of Washington DC
```
{
    'name': 'washington dc',
    'data_type': 'file',
    'value': '.../district_of_columbia_boundary.geojson'
}
```
Bounding box that represents complete extent of Augusta County, VA.
```
{
    'name': 'augusta county, va',
    'data_type': 'bbox',
    'value': [-79.53330839559011, 37.88158664008918, -78.74939547673364, 38.477678774587105]
}
```

### Find all lidar collections that intersect provided locations (using USIAEI)

USIAEI lists all lidar collections, but sometimes paths to EPT (Entwine Point Tiles) files are not included. Another source of these EPT files is NOAA's Digital Access Viewer (DAV). First step is to webscrape all of NOAA's collection names and corresponding EPT files if any and cross walk the records from USIAEI and NOAA.

In [ ]:
find_lidar = lapis.SearchLidarInventory(locations_dict)
locations, collections = find_lidar.get_lidar_collections()

### Result(s)

In [ ]:
locations[['name', 'geometry']]

In [ ]:
collections.head(3)

# Map and explore lidar collections <a name="item"></a>

In [ ]:
m = leafmap.Map(center=[39, -77], zoom=4)
m.add_tile_layer(
    url="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
    name="Google Satellite",
    attribution="Google",
)
# style = {
#     'color': "red",
#     'fillOpacity': 0.5}

m.add_gdf(collections, layer_name="collections", style={'color': 'blue'})
m.add_gdf(locations, layer_name="locations", style={'color': 'red','fillColor': '#3366cc'})
m

### Cross-reference NOAA DAV - by web scraping 

In [ ]:
noaa_dav_gdf = noaa_dav.scrape_digital_coast_repo()

### Parse lidar collections and extract EPT (if any)

In [ ]:
collections = lapis.ProcessLidarCollections(collections, locations, noaa_dav_gdf).execute()

Notable columns in this dataframe:
- `Title`: Lidar collection title
- `collectionyear`: from USIEI -- year of lidar collection
- `ept_usiaei`: EPT file path from USIAEI if present
- `noaa_id`: If a USIEI collection is present in NOAA DAV repo then the unique will be present
- `ID #`: same as `noaa_id` for cross-walk
- `ept_noaa`: EPT file paths pulled from NOAA DAV
- `ept`: reconciled EPT urls -- this cross-references USIAEI and NOAA DAV and used to retrieve point clouds
- `ept_crs`: CRS for EPT files which is needed for on-the-fly transformation
- `ept_gdf`: GeoDataframe of the original location feature reprojected to the native CRS of the EPT file

In [ ]:
collections[['name', 'Title', 'collectionyear', 'collectiondate', 'meets3dep', 'Status']].head(3)

EPT files and/or lidar point clouds are stored in their native projection system. In the collections, geodataframe. The `ept_gdf` column stores the location geodataframe transformed to the native EPT CRS. 

## Enter the index number to extract `ept` url path and `ept_bbox` bounding box to retrieve the lidar point clouds using PDAL

In [ ]:
index = 0
collection = collections.iloc[[index]]
collection[['Title', 'name']]

In [ ]:
count, arrays, metadata, demo_las = lapis.fetch_lidar(collection, geometry_method='polygon', write=True)

# Visualizing the point cloud data

In [ ]:
leafmap.view_lidar(demo_las, cmap='terrain', backend='pyvista')

In [ ]:
array = arrays[0] # 0 is for data and 1 is for datatype + column names

print('no.of points / count: ', count)
print('no. arrays: ', len(arrays))
# print('metadata: ', metadata)

In [ ]:
timestamps = gps_time.LidarTimestamps(arrays[0]).get_timestamps()

In [ ]:
sorted_timestamps = sorted(timestamps)

In [ ]:
print(f"Total Duration of the Collection: {collection.collectiondate.values[0]} \nlidar collection start: {sorted_timestamps[0]} UTC & ends: {sorted_timestamps[-1]} UTC")

### Pending workflow to extract rating curve at the time of the lidar collection

Substitue current rating curve created for colesville gage

In [ ]:
site = "01650500"

What rating type to pick? exsa or base
`https://rconnect.usgs.gov/dataRetrieval/reference/readNWISrating.html`

In [ ]:
rating_data = nwis.get_ratings(site=site, file_type="base") # or base?
(rating_data)

In [ ]:
rating_data = nwis.get_ratings(site=site, file_type="corr") # or base?
(rating_data)

In [ ]:
rating_data = nwis.get_ratings(site=site, file_type="exsa") # or base?
(rating_data)

In [ ]:
rating_data = nwis.get_ratings(site=site, file_type="exsa") # or base?
rating_data = flood_stats.format_rating_data(rating_data)

In [ ]:
(rating_data)

In [ ]:
data = nwis.get_ratings(site=site, file_type="base")

Pull peak flow stats from StreamStats API and convert it into a dataframe

In [ ]:
peak_flow_stats = flood_stats.PeakFlowStatistics(site).process()

In [ ]:
peak_flow_stats[['pfs_aep_name', 'pfs_flow_cfs']]

Cross-reference AEP flow values with flow values from rating curve and find corresponding stage heights

In [ ]:
peak_flow_stats = flood_stats.FlowToStage(
    peak_flow_stats, rating_curve
    ).find_stage_associated_with_aep_flow()

- `pfs_aep_name`: peak flow statistic (PFS) annual exceedance probability (AEP) name
- `pfs_aep_code`: PFS AEP alphanumeric code
---
- `pfs_flow_cfs`: PFS AEP flow/discharge in cubic feet per second
- `rc_flow_cfs`: rating curve (stage versus flow/discharge in cubic feet per second)
- `rc_stage_ft`: rating curve stage height in feet
---
- `pfs_flow_cms`: PFS AEP flow/discharge in cubic meters per second
- `rc_flow_cms`: rating curve (stage versus flow/discharge in cubic meters per second)
- `rc_stage_m`: rating curve stage height in meters

In [ ]:
peak_flow_stats[['pfs_aep_name', 'pfs_flow_cms', 'rc_flow_cms', 'rc_stage_m']]

to do list:
- add raster and hand grid
- visualize raster and hand grid